
---

### 알고리즘 설명

이 노트북은 **협업 필터링**의 한 종류인 **ALS(Alternating Least Squares)** 알고리즘을 사용하여 영화 추천 시스템을 구축합니다.

#### 1. 데이터 로드 및 탐색 (Part 1: Data ETL and Data Exploration)

* **데이터 소스**: 모든 데이터(movies.csv, ratings.csv, links.csv, tags.csv)는 **Databricks Unity Catalog Volume**에서 로드됩니다. 이는 Databricks 환경에서 데이터를 안전하고 효율적으로 관리하는 방법입니다.
* **SparkSession 초기화**: Spark 작업을 시작하기 위해 `SparkSession` 객체를 생성합니다.
* **데이터 로드**: `spark.read.load()`를 사용하여 CSV 파일을 Spark DataFrame으로 읽어 들입니다. `header=True`는 첫 행이 컬럼 이름임을 나타내고, `inferSchema=True`는 Spark가 컬럼의 데이터 타입을 자동으로 추론하도록 합니다.
* **초기 데이터 분석**:
    * `show(5)`를 통해 각 DataFrame의 상위 몇 개 행을 확인하여 데이터의 구조를 파악합니다.
    * 사용자 및 영화별 최소 평가 횟수를 계산하여 데이터의 희소성(sparsity)을 간접적으로 파악합니다.
    * 단 한 명의 사용자에게만 평가된 영화의 수를 계산하여 데이터 분포를 이해합니다.
    * `movies`와 `ratings` DataFrame을 **left outer join**하여 아직 평가되지 않은 영화 목록을 찾습니다. 이는 추천 시스템의 초기 단계에서 사용자에게 새로운 영화를 제안하는 데 유용할 수 있습니다.
* **장르 분석 및 원-핫 인코딩**:
    * `genres` 컬럼을 `|` 기준으로 분리하고 `explode` 함수를 사용하여 각 장르를 개별 행으로 펼쳐 모든 고유 장르를 식별합니다.
    * 각 영화의 장르를 나타내는 **원-핫 인코딩(one-hot encoding)** 방식을 사용한 DataFrame을 생성합니다. 이는 각 장르가 개별 컬럼이 되고, 해당 영화가 그 장르에 속하면 1, 아니면 0의 값을 가집니다.

#### 2. ALS 모델 학습 (Part 2: Spark ALS based approach for training model)

* **데이터 전처리 및 분할**:
    * 추천 모델 학습에 필요한 `userId`, `movieId`, `rating` 컬럼을 선택하고 적절한 데이터 타입(정수, 실수)으로 캐스팅합니다.
    * `na.drop()`을 통해 이 세 컬럼 중 하나라도 NULL 값이 있는 행은 제거하여 모델 학습의 안정성을 확보합니다.
    * 데이터셋을 **훈련(60%), 검증(20%), 테스트(20%) 세트로 무작위 분할**합니다. `randomSplit` 함수는 재현성을 위해 `seed` 값을 사용할 수 있습니다.
    * 분할된 데이터셋은 성능 향상을 위해 Spark 캐시에 저장됩니다(`cache()`).
* **ALS 알고리즘**:
    * ALS는 협업 필터링 알고리즘 중 하나로, 사용자-항목 평점 행렬을 두 개의 저랭크(low-rank) 행렬(사용자 잠재 요인 행렬, 항목 잠재 요인 행렬)의 곱으로 분해합니다.
    * `pyspark.ml.recommendation.ALS` 클래스를 사용합니다. 이는 Spark의 최신 MLlib API로, DataFrame 기반으로 동작합니다.
    * **하이퍼파라미터**:
        * `rank` (잠재 요인 수): 사용자 및 항목의 특징을 나타내는 잠재 요인의 차원입니다. 너무 낮으면 모델이 충분히 복잡한 관계를 학습하지 못하고, 너무 높으면 과적합될 수 있습니다.
        * `maxIter` (최대 반복 횟수): ALS 알고리즘의 반복 횟수입니다.
        * `regParam` (정규화 파라미터): 과적합을 방지하기 위한 정규화 강도입니다. 값이 클수록 모델 복잡성이 줄어듭니다.
        * `userCol`, `itemCol`, `ratingCol`: DataFrame에서 사용자, 항목, 평점 정보를 나타내는 컬럼 이름입니다.
        * `coldStartStrategy="drop"`: 학습 데이터에 없는 새로운 사용자나 항목에 대한 예측 시 NaN 값을 반환하지 않고 해당 행을 드롭합니다. 이는 평가 지표 계산 시 NaN으로 인한 오류를 방지하는 데 중요합니다.
* **모델 선택 및 평가 (하이퍼파라미터 튜닝)**:
    * `train_ALS_df` 함수는 `ranks`와 `reg_params`의 다양한 조합에 대해 ALS 모델을 학습하고 검증 세트에서 **RMSE(Root Mean Squared Error)**를 계산합니다.
    * **그리드 탐색**: 가능한 모든 하이퍼파라미터 조합을 시도하여 가장 낮은 RMSE를 달성하는 모델을 찾습니다.
    * 각 조합의 RMSE를 출력하고, 가장 낮은 RMSE를 가진 모델의 `rank`와 `regParam`을 "최적의 모델"로 기록합니다.
* **학습 곡선 플로팅**:
    * `plot_als_learning_curve` 함수는 특정 `rank`와 `regParam` 값에서 `maxIter`의 변화에 따른 검증 RMSE를 시각화합니다.
    * 이를 통해 모델이 충분히 수렴했는지, 또는 더 많은 반복이 필요한지 등을 판단할 수 있습니다.

#### 3. 모델 테스트 및 추천 생성

* **최종 모델 평가**: 최적의 하이퍼파라미터로 학습된 `final_model`을 사용하여 **테스트 세트**에 대한 예측을 수행하고 최종 RMSE를 계산합니다. 테스트 세트의 RMSE는 모델이 이전에 보지 못한 데이터에 대해 얼마나 잘 일반화되는지를 보여줍니다.
* **추천 생성**:
    * `final_model.recommendForAllUsers(10)`: 각 사용자에게 평점 10개를 예측하여 상위 10개 영화를 추천합니다.
    * `final_model.recommendForAllItems(10)`: 각 영화에 대해 평점 10개를 예측하여 상위 10명 사용자를 추천합니다. 이 기능은 특정 영화를 좋아할 만한 사용자를 찾는 데 사용될 수 있습니다.
* `display()`: Databricks 환경에서 DataFrame을 시각적으로 편리하게 테이블 형태로 출력하는 데 사용됩니다.

이 코드는 Spark MLlib의 ALS를 사용하여 기본적인 협업 필터링 기반 추천 시스템을 구축하고 평가하는 표준적인 파이프라인을 잘 보여줍니다. 특히, Databricks Unity Catalog Volume을 활용하여 데이터 로드를 통합한 점이 특징입니다.


In [0]:
# Databricks notebook source
# MAGIC %md
# MAGIC ### Spark HW3 Moive Recommendation (Unity Catalog Version)
# MAGIC In this notebook, we will use an Alternating Least Squares (ALS) algorithm with Spark APIs to predict the ratings for the movies in [MovieLens small dataset](https://grouplens.org/datasets/movielens/latest/), loaded from a Unity Catalog Volume.
# MAGIC

# COMMAND ----------

import numpy as np # 숫자 배열 및 수학 연산을 위한 라이브러리
import pandas as pd # 데이터 조작 및 분석을 위한 라이브러리 (주로 데이터 시각화 보조 목적)
import seaborn as sns # 데이터 시각화 라이브러리 (matplotlib 기반)
import matplotlib.pyplot as plt # 플로팅 및 그래프 생성을 위한 라이브러리
import math # 수학 함수 (예: isinf, isnan) 사용을 위한 라이브러리

# COMMAND ----------

# MAGIC %md
# MAGIC ## Part1: Data ETL and Data Exploration
# MAGIC ## Part1: 데이터 ETL 및 데이터 탐색

# COMMAND ----------

from pyspark.sql import SparkSession # Spark 애플리케이션의 진입점인 SparkSession 임포트
spark = SparkSession \
    .builder \
    .appName("movie_analysis_uc") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()
# SparkSession 객체를 생성합니다.
# .appName("movie_analysis_uc"): Spark 애플리케이션의 이름을 설정합니다.
# .config("spark.some.config.option", "some-value"): 추가적인 Spark 설정을 할 수 있습니다. 여기서는 예시입니다.
# .getOrCreate(): SparkSession이 이미 실행 중이면 기존 것을 반환하고, 없으면 새로 생성합니다.

# COMMAND ----------

# Define the base path for your data in Unity Catalog Volumes
uc_volume_path = "/Volumes/1dt_team8_managed/movie/movie"
# Unity Catalog Volume에 저장된 데이터의 기본 경로를 정의합니다.
# 이 경로는 Databricks Unity Catalog에서 설정된 볼륨의 경로와 일치해야 합니다.

# Load data from Unity Catalog Volume
try:
    movies = spark.read.table("`1dt_team8_managed`.`movielens-small`.movies")
    ratings = spark.read.table("`1dt_team8_managed`.`movielens-small`.ratings")
    links = spark.read.table("`1dt_team8_managed`.`movielens-small`.links")
    tags = spark.read.table("`1dt_team8_managed`.`movielens-small`.tags")
    print("Data loaded successfully from Unity Catalog Volume.")
except Exception as e:
    print(f"Error loading data from Unity Catalog Volume: {e}")
    print(f"Please ensure CSV files (movies.csv, ratings.csv, links.csv, tags.csv) exist in {uc_volume_path}")
    # dbutils.fs.ls(uc_volume_path) # Uncomment to list files in the volume for debugging
# Unity Catalog Volume에서 Spark DataFrame으로 로드합니다.
# inferSchema=True: Spark가 데이터 타입(스키마)을 자동으로 추론하도록 합니다.
# 데이터 로드 중 오류가 발생하면 예외 처리를 통해 메시지를 출력합니다.

# COMMAND ----------

movies.show(5) # movies DataFrame의 상위 5개 행을 출력합니다.
#display(movies.limit(5)) # Databricks 환경에서 테이블 형태로 데이터를 시각적으로 표시하는 데 사용됩니다. (주석 처리됨)

# COMMAND ----------

ratings.show(5) # ratings DataFrame의 상위 5개 행을 출력합니다.

# COMMAND ----------

# Ensure ratings DataFrame is loaded before proceeding
if 'ratings' in locals(): # ratings DataFrame이 성공적으로 로드되었는지 확인합니다.
    tmp1 = ratings.groupBy("userID").count().toPandas()['count'].min()
    tmp2 = ratings.groupBy("movieId").count().toPandas()['count'].min()
    print('For the users that rated movies and the movies that were rated:')
    print('Minimum number of ratings per user is {}'.format(tmp1)) # 각 사용자별 최소 평가 횟수
    print('Minimum number of ratings per movie is {}'.format(tmp2)) # 각 영화별 최소 평가 횟수
else:
    print("Ratings DataFrame not loaded. Please check the data loading step.")
# 사용자 및 영화별 최소 평가 횟수를 계산하여 출력합니다.
# .groupBy("userID").count(): 각 사용자 ID별 평가 횟수를 집계합니다.
# .toPandas(): Spark DataFrame을 Pandas DataFrame으로 변환합니다.
# ['count'].min(): Pandas DataFrame에서 'count' 컬럼의 최소값을 추출합니다.

# COMMAND ----------

if 'ratings' in locals(): # ratings DataFrame이 성공적으로 로드되었는지 확인합니다.
    tmp1 = sum(ratings.groupBy("movieId").count().toPandas()['count'] == 1)
    tmp2 = ratings.select('movieId').distinct().count()
    print('{} out of {} movies are rated by only one user'.format(tmp1, tmp2))
else:
    print("Ratings DataFrame not loaded.")
# 단 한 명의 사용자에게만 평가된 영화의 수를 계산하여 출력합니다.

# COMMAND ----------

# MAGIC %md
# MAGIC ## Part 1: Spark SQL and OLAP
# MAGIC ## Part 1: Spark SQL 및 OLAP

# COMMAND ----------

# MAGIC %md ### The number of Users
# MAGIC ### 사용자 수

# COMMAND ----------

if 'ratings' in locals(): # ratings DataFrame이 성공적으로 로드되었는지 확인합니다.
    tmp_q1 = ratings.select('userid').distinct().count() # 'userid' 컬럼의 고유한 값의 개수를 세어 총 사용자 수를 계산합니다.
    print('There totally have {} users'.format(tmp_q1))
else:
    print("Ratings DataFrame not loaded.")

# COMMAND ----------

# MAGIC %md ### The number of Movies
# MAGIC ### 영화 수

# COMMAND ----------

if 'movies' in locals(): # movies DataFrame이 성공적으로 로드되었는지 확인합니다.
    tmp_q2 = movies.select('movieid').distinct().count() # 'movieid' 컬럼의 고유한 값의 개수를 세어 총 영화 수를 계산합니다.
    print('There totally have {} movies'.format(tmp_q2))
else:
    print("Movies DataFrame not loaded.")

# COMMAND ----------

# MAGIC %md ### How many movies are rated by users? List movies not rated before
# MAGIC ### 사용자에 의해 몇 개의 영화가 평가되었는가? 평가되지 않은 영화 목록을 출력하라.

# COMMAND ----------

from pyspark.sql.functions import col, isnan # col 함수를 임포트합니다. isnan은 여기서는 사용되지 않습니다.

if 'movies' in locals() and 'ratings' in locals(): # movies 및 ratings DataFrame이 성공적으로 로드되었는지 확인합니다.
    tmp_q3_rated_count = ratings.select('movieid').distinct().count() # 평가된 고유 영화의 수를 계산합니다.
    total_movies_count = movies.select('movieid').distinct().count() # 총 고유 영화의 수를 계산합니다.
    print('{} movies have not been rated'.format(total_movies_count - tmp_q3_rated_count)) # 평가되지 않은 영화의 수를 계산하여 출력합니다.

    # Join movies with ratings to find movies with no ratings
    # Ensure movieId columns are of the same type if join issues occur. inferSchema should help.
    movies_with_ratings = movies.join(ratings, movies.movieId == ratings.movieId, "left_outer")
    # movies DataFrame과 ratings DataFrame을 movieId를 기준으로 LEFT OUTER JOIN합니다.
    # LEFT OUTER JOIN은 왼쪽 DataFrame(movies)의 모든 레코드를 포함하고, 오른쪽 DataFrame(ratings)에서 일치하는 레코드를 가져옵니다.
    # 일치하는 레코드가 없으면 ratings 테이블의 컬럼은 NULL이 됩니다.

    # Select movies where the rating (from the ratings table) is null
    unrated_movies = movies_with_ratings.where(ratings.rating.isNull()) \
                                      .select(movies.movieId, movies.title).distinct() # Added distinct
    # 조인 결과에서 ratings.rating이 NULL인 행을 필터링하여 평가되지 않은 영화를 찾습니다.
    # 그리고 movieid와 title을 선택한 후 중복을 제거합니다.
    print("\nList of movies not rated before:")
    unrated_movies.show() # 평가되지 않은 영화 목록을 출력합니다.
else:
    print("Movies or Ratings DataFrame not loaded.")

# COMMAND ----------

# MAGIC %md ### List Movie Genres
# MAGIC ### 영화 장르 목록

# COMMAND ----------

import pyspark.sql.functions as f # Spark SQL 함수에 대한 별칭 'f'를 임포트합니다.

if 'movies' in locals(): # movies DataFrame이 성공적으로 로드되었는지 확인합니다.
    # Explode the genres string into an array, then explode the array into multiple rows
    all_genres_df = movies.withColumn("genre_array", f.split(col("genres"), "\|")) \
                          .select(f.explode(col("genre_array")).alias("genre")) \
                          .distinct()
    # 'genres' 컬럼의 문자열을 '|' 기준으로 분리하여 배열('genre_array')로 만듭니다.
    # f.explode()를 사용하여 배열의 각 요소를 개별 행으로 펼치고, 이를 'genre'라는 새 컬럼으로 명명합니다.
    # .distinct(): 고유한 장르 목록을 얻기 위해 중복을 제거합니다.

    distinct_genres_list = [row.genre for row in all_genres_df.collect()]
    # all_genres_df의 모든 행을 드라이버 노드로 가져와(collect()) 'genre' 컬럼 값을 리스트로 변환합니다.
    hashset = set(distinct_genres_list) 
    # 리스트를 set으로 변환하여 다시 한번 고유성을 보장하고 빠른 조회를 가능하게 합니다.

    print("Distinct genres found:")
    print(hashset) # 고유한 장르 집합을 출력합니다.
    print("Total number of distinct genres: {}".format(len(hashset))) # 총 고유 장르 수를 출력합니다.
else:
    print("Movies DataFrame not loaded.")

# COMMAND ----------

# MAGIC %md ### Movie for Each Category
# MAGIC This part creates a one-hot encoded representation for genres.
# MAGIC The original implementation was a bit complex. A more PySpark-idiomatic way is to use pivot or conditional aggregation.
# MAGIC ### 각 카테고리별 영화
# MAGIC 이 부분은 장르에 대한 원-핫 인코딩 표현을 생성합니다.
# MAGIC 원래 구현은 다소 복잡했습니다. PySpark에 더 적합한 방법은 피벗(pivot) 또는 조건부 집계(conditional aggregation)를 사용하는 것입니다.

# COMMAND ----------

from pyspark.sql.functions import expr, when # expr과 when 함수를 임포트합니다.

if 'movies' in locals() and 'hashset' in locals() and len(hashset) > 0: # movies DataFrame과 hashset이 유효한지 확인합니다.
    q5_base = movies.select("movieid", "title", "genres") # movieid, title, genres 컬럼만 선택하여 새 DataFrame을 생성합니다.

    # Create a column for each genre, with 1 if the movie has that genre, 0 otherwise
    # This approach is more scalable and idiomatic in Spark than manual list iteration for DataFrame construction
    genre_expressions = [
        when(col("genres").rlike(genre.replace("(", "\\(").replace(")", "\\)")), 1).otherwise(0).alias(genre)
        for genre in hashset if genre != '(no genres listed)' # Handle special characters if any in genre names for alias
    ]
    # hashset에 있는 각 장르에 대해 원-핫 인코딩 컬럼을 생성합니다.
    # `when(col("genres").rlike(genre), 1).otherwise(0).alias(genre)`:
    # - `col("genres").rlike(genre)`: 'genres' 컬럼이 현재 장르 문자열을 포함하는지 정규 표현식으로 확인합니다.
#          (no genres listed)와 같은 특수 문자가 장르 이름에 포함될 경우 RLIKE 패턴에 영향을 주지 않도록 이스케이프 처리합니다.
# - `when(조건, 1).otherwise(0)`: 조건이 참이면 1, 아니면 0을 반환합니다.
# - `.alias(genre)`: 새로 생성될 컬럼의 이름을 해당 장르 이름으로 지정합니다.
# '(no genres listed)'는 별도로 처리됩니다.

    # Add (no genres listed) separately if it exists, ensuring valid alias
    if '(no genres listed)' in hashset:
        genre_expressions.append(
            when(col("genres") == '(no genres listed)', 1).otherwise(0).alias("no_genres_listed")
        )
# '(no genres listed)' 장르가 존재하면 "no_genres_listed"라는 별칭으로 별도 컬럼을 추가합니다.

    if genre_expressions: # 생성할 장르 표현식이 하나라도 있는지 확인합니다.
        tmp_q5 = q5_base.select(col("movieid"), col("title"), *genre_expressions)
        # movieid, title 컬럼과 생성된 모든 장르 원-핫 인코딩 컬럼을 포함하는 DataFrame을 생성합니다.
        print("\nMovies with one-hot encoded genres:")
        tmp_q5.show() # 결과 DataFrame을 출력합니다.

        # Example: List "Drama" movies
        # Adjust the alias if 'Drama' was changed (e.g., due to special characters)
        drama_alias = "Drama" # 'Drama' 장르의 별칭을 설정합니다.
        if drama_alias in tmp_q5.columns: # 'Drama' 컬럼이 DataFrame에 존재하는지 확인합니다.
            tmp_drama = tmp_q5.filter(col(drama_alias) == 1).select("movieid", "title")
            # 'Drama' 컬럼 값이 1인 영화만 필터링하여 Drama 장르 영화를 추출합니다.
            print("\n{} movies are Drama, they are:".format(tmp_drama.count()))
            tmp_drama.show() # Drama 영화 목록을 출력합니다.
        else:
            print(f"\nColumn '{drama_alias}' not found. Available columns: {tmp_q5.columns}")
    else:
        print("\nNo genres found to process for one-hot encoding.")

else:
    print("Movies DataFrame or genre set not available for one-hot encoding.")


# COMMAND ----------

# MAGIC %md
# MAGIC ## Part2: Spark ALS based approach for training model
# MAGIC We will use an RDD-based API from [pyspark.mllib](https://spark.apache.org/docs/2.1.1/mllib-collaborative-filtering.html) (older API) and then switch to MLlib's DataFrame-based API for ALS.
# MAGIC
# MAGIC Note: The original code uses `sc.textFile` which is for RDDs. For consistency and modern Spark, it's often better to read into a DataFrame first and then convert to RDD if absolutely necessary. However, we'll keep `sc.textFile` for this part to match the original structure, but load from the UC Volume.
# MAGIC ## Part2: 모델 학습을 위한 Spark ALS 기반 접근 방식
# MAGIC [pyspark.mllib](https://spark.apache.org/docs/2.1.1/mllib-collaborative-filtering.html) (구형 API)의 RDD 기반 API를 사용한 다음, MLlib의 DataFrame 기반 ALS API로 전환합니다.
# MAGIC
# MAGIC 참고: 원본 코드는 RDD용인 `sc.textFile`을 사용합니다. 일관성과 최신 Spark를 위해 일반적으로 DataFrame으로 먼저 읽은 다음, 반드시 필요한 경우에만 RDD로 변환하는 것이 좋습니다. 하지만, 이 부분에서는 원본 구조와 일치시키기 위해 `sc.textFile`을 유지하지만, UC Volume에서 로드합니다.

# COMMAND ----------

# Load ratings data using Spark DataFrame API from Unity Catalog Volume

# Define the base path for your data in Unity Catalog Volumes
uc_volume_path = "/Volumes/dbricks_jhlee/default/movie"
# Unity Catalog Volume 내의 데이터 기본 경로를 다시 정의합니다 (이전 경로와 다를 수 있음).

# Load ratings data using Spark DataFrame API from Unity Catalog Volume
# This approach is compatible with Databricks Serverless compute.
ratings_file_path_uc = f"{uc_volume_path}/ratings.csv"
from pyspark.sql.functions import col # col 함수를 임포트합니다.

try:
    # 1. Read CSV into a DataFrame
    ratings_df_initial = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(ratings_file_path_uc)
    # ratings.csv 파일을 DataFrame으로 로드합니다. (헤더 포함, 스키마 추론)

    # 2. Select necessary columns and explicitly cast to desired types.
    #    The original RDD code effectively took (userId, movieId, rating).
    #    We drop rows if any of these key columns are null after casting.
    #    This DataFrame will be directly used for splitting.
    ratings_for_split_df = ratings_df_initial.select(
        col("userId").cast("integer"),
        col("movieId").cast("integer"),
        col("rating").cast("float")
    ).na.drop() # Drop rows where userId, movieId, or rating is null
    # userId, movieId는 정수형으로, rating은 실수형으로 캐스팅합니다.
    # .na.drop(): userId, movieId, rating 중 하나라도 NULL 값인 행은 제거합니다.
    # 이 DataFrame은 모델 학습을 위한 데이터 분할에 직접 사용됩니다.

    print("Ratings data loaded into DataFrame for splitting:")
    ratings_for_split_df.show(5) # 분할 준비된 ratings DataFrame의 상위 5개 행을 출력합니다.
    ratings_for_split_df.printSchema() # DataFrame의 스키마를 출력합니다.

except Exception as e:
    print(f"Error loading ratings.csv into DataFrame from {ratings_file_path_uc}: {e}")
    ratings_for_split_df = None # 오류 발생 시 ratings_for_split_df를 None으로 설정합니다.

# COMMAND ----------

# MAGIC %md Now we split the data into training/validation/testing sets using a 6/2/2 ratio.
# MAGIC 이제 데이터를 훈련/검증/테스트 세트로 6/2/2 비율로 분할합니다.

# COMMAND ----------

if ratings_for_split_df: # ratings_for_split_df가 유효한지 확인합니다.
    # Split the DataFrame directly
    (train_df, validation_df, test_df) = ratings_for_split_df.randomSplit([0.6, 0.2, 0.2], seed=7856)
    # DataFrame을 훈련(60%), 검증(20%), 테스트(20%) 세트로 무작위 분할합니다.
    # seed=7856: 재현성을 위해 시드 값을 설정합니다.

    # Cache the DataFrames for performance
    train_df.cache() # 훈련 데이터를 캐시하여 반복적인 접근 시 성능을 향상시킵니다.
    validation_df.cache() # 검증 데이터를 캐시합니다.
    test_df.cache() # 테스트 데이터를 캐시합니다.

    print("Data split into Training, Validation, and Test DataFrames.")
    print(f"Training DataFrame count: {train_df.count()}") # 훈련 데이터셋의 행 수를 출력합니다.
    print(f"Validation DataFrame count: {validation_df.count()}") # 검증 데이터셋의 행 수를 출력합니다.
    print(f"Test DataFrame count: {test_df.count()}") # 테스트 데이터셋의 행 수를 출력합니다.
    print("\nSchema of training DataFrame:")
    train_df.printSchema() # 훈련 데이터셋의 스키마를 출력합니다.
    train_df.show(3) # 훈련 데이터셋의 상위 3개 행을 출력합니다.
else:
    print("ratings_for_split_df DataFrame not available for splitting.")
    # Define empty DataFrames or handle error appropriately if needed downstream
    train_df, validation_df, test_df = [spark.createDataFrame([], ratings_for_split_df.schema if ratings_for_split_df else spark.read.format("csv").load(ratings_file_path_uc).schema) for _ in range(3)]
# ratings_for_split_df가 없으면 빈 DataFrame을 생성하여 하위 코드에서 오류가 발생하지 않도록 합니다.

# COMMAND ----------

# MAGIC %md
# MAGIC ### ALS Model Selection and Evaluation
# MAGIC
# MAGIC With the ALS model, we can use a grid search to find the optimal hyperparameters.
# MAGIC We will now use the DataFrame-based API for ALS (`pyspark.ml.recommendation.ALS`), which is generally preferred.
# MAGIC ### ALS 모델 선택 및 평가
# MAGIC
# MAGIC ALS 모델을 사용하면 그리드 탐색을 통해 최적의 하이퍼파라미터를 찾을 수 있습니다.
# MAGIC 이제 일반적으로 선호되는 DataFrame 기반 ALS API(`pyspark.ml.recommendation.ALS`)를 사용합니다.

# COMMAND ----------

from pyspark.ml.recommendation import ALS # ALS(Alternating Least Squares) 추천 알고리즘을 임포트합니다.
from pyspark.ml.evaluation import RegressionEvaluator # 회귀 모델 평가를 위한 RegressionEvaluator를 임포트합니다 (RMSE 계산).

def train_ALS_df(train_data, validation_data, num_iters, reg_params, ranks_list): # ranks를 ranks_list로 이름 변경
    min_error = float('inf') # 최소 RMSE를 저장할 변수를 무한대로 초기화합니다.
    best_rank = -1 # 최적의 랭크 값을 저장할 변수를 초기화합니다.
    best_regularization = 0 # 최적의 정규화 파라미터 값을 저장할 변수를 초기화합니다.
    best_model = None # 최적의 모델을 저장할 변수를 초기화합니다.

    for rank_val in ranks_list: # rank_val 사용
        for reg in reg_params: # 주어진 정규화 파라미터 목록을 순회합니다.
            als = ALS(rank=rank_val, maxIter=num_iters, regParam=reg,
                      userCol="userId", itemCol="movieId", ratingCol="rating",
                      coldStartStrategy="drop", # 중요한 설정: 새로운 사용자/항목에 대한 예측 시 NaN 방지 (해당 행 드롭)
                      seed=42) # 재현성을 위해 시드 값을 설정합니다.
            try:
                model = als.fit(train_data) # 훈련 데이터로 ALS 모델을 학습시킵니다.
                predictions = model.transform(validation_data) # 검증 데이터에 대한 예측을 수행합니다.
                
                # Remove NaNs from predictions if any, as evaluator cannot handle them
                predictions_cleaned = predictions.filter(predictions.prediction.isNotNull())
                # 예측 결과에서 NaN (Not a Number) 값을 포함하는 행을 제거합니다. Evaluator는 NaN을 처리할 수 없습니다.
                
                evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
                # RMSE(Root Mean Squared Error)를 평가 지표로 사용하는 RegressionEvaluator를 생성합니다.
                
                if predictions_cleaned.count() == 0:
                    rmse_error = float('inf') # Or handle as a failed case
                    print(f"Warning: No valid predictions for rank={rank_val}, reg={reg}. All predictions were NaN or validation set was empty after coldStartStrategy.")
                else:
                    rmse_error = evaluator.evaluate(predictions_cleaned) # 클리닝된 예측으로 RMSE를 계산합니다.

                print('Rank = {}, Regularization = {}: Validation RMSE = {}'.format(rank_val, reg, rmse_error))
                
                if rmse_error < min_error: # 현재 RMSE가 이전 최소 RMSE보다 작으면
                    min_error = rmse_error # 최소 RMSE를 업데이트합니다.
                    best_rank = rank_val # 최적 랭크를 업데이트합니다.
                    best_regularization = reg # 최적 정규화 파라미터를 업데이트합니다.
                    best_model = model # 최적 모델을 저장합니다.
            except Exception as e:
                print(f"Error training ALS with rank={rank_val}, reg={reg}: {e}")
                continue # 다음 하이퍼파라미터 조합으로 넘어갑니다.

    if best_model: # 최적 모델이 존재하면
        print('\nThe best model has {} latent factors and regularization = {}'.format(best_rank, best_regularization))
    else: # 최적 모델을 찾지 못했으면
        print('\nCould not find a best model. All training attempts failed or produced no valid predictions.')
    return best_model # 최적 모델을 반환합니다.

# COMMAND ----------

if 'train_df' in locals() and 'validation_df' in locals(): # 훈련 및 검증 DataFrame이 유효한지 확인합니다.
    num_iterations = 10 # 각 모델 학습에 사용할 최대 반복 횟수
    ranks_param = [6, 8, 10, 12] # 테스트할 랭크(잠재 요인 수) 목록
    reg_params_param = [0.05, 0.1, 0.2, 0.4] # 테스트할 정규화 파라미터 목록
    import time # 시간 측정을 위한 time 모듈 임포트

    start_time = time.time() # 모델 튜닝 시작 시간 기록
    # train_ALS_df 함수를 호출하여 최적의 ALS 모델을 찾고, 그 결과를 final_model에 저장합니다.
    # 함수 정의를 수정하여 best_rank 및 best_regularization도 반환하도록 변경하지 않았다면,
    # 이 호출은 오직 모델 객체만 반환합니다.
    # 기존 함수 train_ALS_df가 best_model만 반환하므로, 해당 함수를 호출하고 이 값만 받습니다.
    # 이후 plot_als_learning_curve에서 필요한 best_reg와 best_rnk는 임시로 설정하거나,
    # train_ALS_df 함수가 이 값들도 반환하도록 수정해야 합니다.
    # 여기서는 단순히 final_model만 저장합니다.
    # 아래 `plot_als_learning_curve` 호출 시에 `final_model`이 직접 rank/regParam 속성을 가지지 않으므로
    # train_ALS_df를 수정하여 최적의 rank와 reg_param을 반환하도록 했습니다.
    final_model, best_rnk_found, best_reg_found = train_ALS_df(train_df, validation_df, num_iterations, reg_params_param, ranks_param)
    print('Total Runtime for Hyperparameter Tuning: {:.2f} seconds'.format(time.time() - start_time)) # 총 실행 시간 출력
else:
    print("Training and validation DataFrames (train_df, validation_df) are not available.")
    final_model = None # DataFrame이 없으면 final_model을 None으로 설정합니다.


# COMMAND ----------

# MAGIC %md
# MAGIC ### Learning Curve
# MAGIC The original `plot_learning_curve` used scikit-learn's `learning_curve` which is not directly applicable here.
# MAGIC We need to manually iterate through `maxIter` for ALS.
# MAGIC ### 학습 곡선
# MAGIC 원본 `plot_learning_curve`는 scikit-learn의 `learning_curve`를 사용했지만 여기서는 직접 적용할 수 없습니다.
# MAGIC ALS의 `maxIter`를 수동으로 반복해야 합니다.

# COMMAND ----------

import matplotlib.pyplot as plt # 함수 내에서 plt를 사용하므로 임포트 확인
from pyspark.ml.recommendation import ALS # ALS 임포트 확인
from pyspark.ml.evaluation import RegressionEvaluator # Evaluator 임포트 확인

def train_ALS_df(train_data, validation_data, num_iters, reg_params_list, ranks_list):
    min_error = float('inf')
    best_rank_val = -1
    best_reg_param_val = 0.0 # float으로 초기화
    best_model_instance = None

    print(f"\n--- Starting Hyperparameter Tuning for ALS ---")
    print(f"Ranks to test: {ranks_list}")
    print(f"Regularization params to test: {reg_params_list}")
    print(f"Number of iterations for each model: {num_iters}")

    if train_data.count() == 0:
        print("Error: Training data is empty. Cannot train model.")
        return None, -1, 0.0 # 학습 데이터가 비어있으면 None과 기본값 반환
    if validation_data.count() == 0:
        print("Warning: Validation data is empty. RMSE will be Inf or NaN.")


    for rank_val_iter in ranks_list: # 주어진 랭크 목록을 순회합니다.
        for reg_param_iter in reg_params_list: # 주어진 정규화 파라미터 목록을 순회합니다.
            print(f"  Training with Rank: {rank_val_iter}, RegParam: {reg_param_iter}")
            als = ALS(rank=rank_val_iter, maxIter=num_iters, regParam=reg_param_iter,
                      userCol="userId", itemCol="movieId", ratingCol="rating",
                      coldStartStrategy="drop", # NaN 평가 메트릭 방지 (평가할 수 없는 새 사용자/영화 드롭)
                      seed=42) # 재현성을 위한 시드 값
            try:
                model = als.fit(train_data) # 훈련 데이터로 ALS 모델을 학습시킵니다.
                
                if validation_data.count() > 0: # 검증 데이터가 있을 때만 평가를 시도합니다.
                    predictions = model.transform(validation_data) # 검증 데이터에 대한 예측을 수행합니다.
                    predictions_cleaned = predictions.filter(predictions.prediction.isNotNull())
                    # 예측 결과에서 NaN 값을 포함하는 행을 제거합니다.
                    
                    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
                    
                    if predictions_cleaned.count() == 0:
                        # coldStartStrategy="drop"으로 인해 모든 검증 데이터가 제거된 경우
                        rmse_error = float('inf') 
                        print(f"    Warning: No valid predictions for rank={rank_val_iter}, reg={reg_param_iter} on validation set. All predictions were NaN or validation set became empty.")
                    else:
                        rmse_error = evaluator.evaluate(predictions_cleaned) # RMSE를 계산합니다.
                else: # 검증 데이터가 처음부터 비어있는 경우
                    rmse_error = float('inf')
                    print(f"    Warning: Validation data is empty. RMSE for rank={rank_val_iter}, reg={reg_param_iter} is Inf.")


                print(f"    Rank = {rank_val_iter}, Regularization = {reg_param_iter}: Validation RMSE = {rmse_error}")
                
                # math.isinf 와 math.isnan 을 사용하여 유효한 에러 값인지 확인
                if not math.isinf(rmse_error) and not math.isnan(rmse_error) and rmse_error < min_error:
                    min_error = rmse_error
                    best_rank_val = rank_val_iter
                    best_reg_param_val = reg_param_iter
                    best_model_instance = model
                    print(f"    New best model found! RMSE: {min_error:.4f}, Rank: {best_rank_val}, Reg: {best_reg_param_val}")

            except Exception as e:
                print(f"    Error training ALS with rank={rank_val_iter}, reg={reg_param_iter}: {e}")
                continue # 다음 하이퍼파라미터 조합으로 넘어갑니다.

    if best_model_instance:
        print(f"\n--- Hyperparameter Tuning Finished ---")
        print(f"The best model has {best_rank_val} latent factors and regularization = {best_reg_param_val:.4f} with RMSE = {min_error:.4f}")
    else:
        print(f"\n--- Hyperparameter Tuning Finished ---")
        print("Could not find a best model. All training attempts might have failed or produced invalid RMSEs.")
    
    return best_model_instance, best_rank_val, best_reg_param_val # 최적 모델과 해당 랭크, 정규화 파라미터를 반환합니다.

# COMMAND ----------

import matplotlib.pyplot as plt # 함수 내에서 plt를 사용하므로 임포트 확인
from pyspark.ml.recommendation import ALS # ALS 임포트 확인
from pyspark.ml.evaluation import RegressionEvaluator # Evaluator 임포트 확인

def plot_als_learning_curve(iter_array, train_data, validation_data, reg, rank_val):
    iter_num_plot, rmse_plot = [], [] # 반복 횟수와 RMSE 값을 저장할 리스트를 초기화합니다.

    for iter_val in iter_array: # 주어진 반복 횟수 목록을 순회합니다.
        als = ALS(rank=rank_val, maxIter=iter_val, regParam=reg,
                userCol="userId", itemCol="movieId", ratingCol="rating",
                coldStartStrategy="drop", seed=42) # ALS 모델을 초기화합니다.
        try:
            model = als.fit(train_data) # 훈련 데이터로 모델을 학습시킵니다.
            predictions = model.transform(validation_data) # 검증 데이터에 대한 예측을 수행합니다.
            # 누락된 예측값(NaN)을 가진 행은 평가 전에 제거하거나 처리해야 합니다.
            predictions_cleaned = predictions.filter(predictions.prediction.isNotNull())
            # 예측 결과에서 NaN 값을 제거합니다.

            evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
            
            if predictions_cleaned.count() == 0:
                # 모든 예측이 NaN이거나 validation_data가 coldStartStrategy에 의해 비워진 경우
                rmse_error = float('nan') 
                print(f"Warning: No valid predictions for iterations={iter_val}, rank={rank_val}, reg={reg}. All predictions were NaN or validation set was empty after coldStartStrategy.")
            else:
                rmse_error = evaluator.evaluate(predictions_cleaned) # RMSE를 계산합니다.
            
            print('Iterations = {}, Rank = {}, Regularization = {}: Validation RMSE = {}'.format(iter_val, rank_val, reg, rmse_error))
            
            iter_num_plot.append(iter_val) # 반복 횟수를 리스트에 추가합니다.
            rmse_plot.append(rmse_error) # RMSE 값을 리스트에 추가합니다.
        except Exception as e:
            print(f"Error training ALS for learning curve with iterations={iter_val}, rank={rank_val}, reg={reg}: {e}")
            iter_num_plot.append(iter_val)
            rmse_plot.append(float('nan')) # 오류 발생 시 NaN 추가
            continue

    # 데이터 유효성 검사 (플로팅 전)
    if not any(math.isfinite(x) for x in rmse_plot if isinstance(x, float)): # rmse_plot에 유효한 숫자가 있는지 확인
        print("No valid RMSE values to plot. Learning curve will not be generated.")
        return # 유효한 데이터가 없으면 플로팅하지 않음

    # Plotting
    fig, ax = plt.subplots(figsize=(12, 6))  # 그림과 축 객체를 생성합니다.
    ax.plot(iter_num_plot, rmse_plot, marker='o', linestyle='--') # 학습 곡선을 그립니다.

    ax.set_xlabel("Number of Iterations") # x축 레이블 설정
    ax.set_ylabel("Validation RMSE") # y축 레이블 설정
    ax.set_title(f"ALS Learning Curve (Rank={rank_val}, RegParam={reg})") # 그래프 제목 설정
    ax.set_xticks(iter_array) # x축 눈금을 iter_array 값으로 설정
    ax.grid(True) # 격자 표시

    # Databricks 노트북에서 matplotlib 그림을 표시하려면 display() 함수를 사용합니다.
    display(fig) # Databricks 노트북에 그래프를 표시합니다.

    plt.show # 그래프를 보여줍니다. (Databricks에서는 display(fig)가 이미 표시하므로 필수는 아님)

# COMMAND ----------

if 'train_df' in locals() and 'validation_df' in locals(): # 훈련 및 검증 DataFrame이 유효한지 확인합니다.
    iter_array_plot = [1, 2, 5, 10, 15] # 학습 곡선을 그릴 반복 횟수 목록
    # Use parameters from the best model, or pick specific ones
    # If final_model exists and has these attributes (depends on how it's stored/returned)
    # train_ALS_df 함수가 최적의 rank와 reg_param을 반환하도록 수정했기 때문에 해당 값을 사용합니다.
    best_reg = best_reg_found if 'best_reg_found' in locals() and best_reg_found is not None else 0.2
    best_rnk = best_rnk_found if 'best_rnk_found' in locals() and best_rnk_found is not None else 10
    
    if final_model is not None: # final_model이 성공적으로 학습되었는지 확인합니다.
        print(f"Plotting learning curve for Rank={best_rnk}, RegParam={best_reg}")
        plot_als_learning_curve(iter_array_plot, train_df, validation_df, best_reg, best_rnk)
    else: # final_model이 없으면 예시 파라미터로 학습 곡선을 그립니다.
        print("final_model not available. Plotting with example parameters (Rank=10, Reg=0.2).")
        plot_als_learning_curve(iter_array_plot, train_df, validation_df, 0.2, 10)

else:
    print("Training and validation DataFrames (train_df, validation_df) are not available for plotting learning curve.")

# COMMAND ----------

# MAGIC %md
# MAGIC ### Model testing
# MAGIC And finally, make a prediction and check the testing error using the `final_model`.
# MAGIC ### 모델 테스트
# MAGIC 마지막으로, `final_model`을 사용하여 예측하고 테스트 오류를 확인합니다.

# COMMAND ----------

if final_model and 'test_df' in locals(): # final_model과 test_df가 유효한지 확인합니다.
    predictions_test = final_model.transform(test_df) # 테스트 데이터에 대한 예측을 수행합니다.
    predictions_test_cleaned = predictions_test.filter(predictions_test.prediction.isNotNull())
    # 테스트 예측 결과에서 NaN 값을 제거합니다.

    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    
    if predictions_test_cleaned.count() == 0:
        rmse_test = float('inf')
        print("Warning: No valid predictions on the test set.")
    else:
        rmse_test = evaluator.evaluate(predictions_test_cleaned) # 테스트 세트의 RMSE를 계산합니다.
    
    print("Test Set Root-mean-square error = " + str(rmse_test)) # 테스트 RMSE를 출력합니다.

    # Generate top 10 movie recommendations for each user
    try:
        userRecs = final_model.recommendForAllUsers(10) # 각 사용자에게 상위 10개 영화를 추천합니다.
        print("\nTop 10 movie recommendations for each user (sample):")
        #userRecs.show(5, truncate=False)
        display(userRecs) # Databricks에 추천 결과를 표시합니다.
    except Exception as e:
        print(f"Error generating user recommendations: {e}")


    # Generate top 10 user recommendations for each movie
    try:
        movieRecs = final_model.recommendForAllItems(10) # 각 영화에 대해 상위 10명의 사용자를 추천합니다.
        print("\nTop 10 user recommendations for each movie (sample):")
        #movieRecs.show(5, truncate=False)
        display(movieRecs) # Databricks에 추천 결과를 표시합니다.
    except Exception as e:
        print(f"Error generating movie recommendations: {e}")

else:
    print("Final model or test DataFrame (test_df) is not available for testing.")
